# Training a cat/dog breed classifier
This notebook uses a DenseNet NN in order to train a model which will be able to classify images of cats and dogs and label their breed. 

At the end of the notebook, also inference code is presented for testing purposes of the model.

The images were taken from [The Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/). The framework that it is used for training the model is [fastai](https://www.fast.ai/).

## Installing the prerequisites

In [ ]:
!pip install fastai==2.6.3
!pip install pillow=9.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi -L
import fastai
fastai.__version__

GPU 0: Tesla T4 (UUID: GPU-3eef8008-a2b0-0eb9-b70a-9c9bfcb14d6b)


'2.6.3'

In [ ]:
import PIL
PIL.__version__

'9.1.0'

In [ ]:
from fastai import *
from fastai.vision.all import *
import pandas as pd

## Preparing the dataset

In [ ]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
!tar xfv images.tar.gz

Streaming output truncated to the last 5000 lines.
images/american_bulldog_194.jpg
images/Birman_175.jpg
images/american_bulldog_178.jpg
images/staffordshire_bull_terrier_93.jpg
images/Egyptian_Mau_58.jpg
images/havanese_44.jpg
images/english_cocker_spaniel_97.jpg
images/Ragdoll_111.jpg
images/american_pit_bull_terrier_159.jpg
images/english_setter_134.jpg
images/english_setter_87.jpg
images/wheaten_terrier_198.jpg
images/pug_180.jpg
images/boxer_172.jpg
images/Sphynx_197.jpg
images/german_shorthaired_2.jpg
images/British_Shorthair_9.jpg
images/pug_89.jpg
images/japanese_chin_39.jpg
images/Ragdoll_20.jpg
images/american_bulldog_111.jpg
images/yorkshire_terrier_112.jpg
images/Persian_88.jpg
images/pug_9.jpg
images/Birman_40.jpg
images/British_Shorthair_272.jpg
images/havanese_7.jpg
images/pomeranian_154.jpg
images/beagle_49.jpg
images/British_Shorthair_94.jpg
images/Persian_90.jpg
images/staffordshire_bull_terrier_30.jpg
images/samoyed_113.jpg
images/english_cocker_spaniel_46.jpg
images

In [ ]:
path = Path('.')
Path.BASE_PATH = path
(path/"images").ls()

(#7393) [Path('images/miniature_pinscher_77.jpg'),Path('images/shiba_inu_163.jpg'),Path('images/chihuahua_24.jpg'),Path('images/Maine_Coon_127.jpg'),Path('images/Ragdoll_193.jpg'),Path('images/havanese_131.jpg'),Path('images/keeshond_171.jpg'),Path('images/havanese_152.jpg'),Path('images/leonberger_62.jpg'),Path('images/Egyptian_Mau_71.jpg')...]

In [ ]:
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images", bs=64)

## Training the model
A DenseNet was finally preferred over a ResNet as it has equivallent performance, but it is much more efficient in terms of space.

In [ ]:
from fastai.callback.fp16 import *
# learn = vision_learner(dls, resnet18, metrics=error_rate).to_fp16()
learn = vision_learner(dls, densenet121, metrics=error_rate).to_fp16()
learn.fine_tune(12, freeze_epochs=3)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,2.046983,0.460978,0.140054,01:15
1,0.778319,0.312189,0.100135,01:14
2,0.447328,0.285783,0.086604,01:12


epoch,train_loss,valid_loss,error_rate,time
0,0.262040,0.223030,0.072395,01:15
1,0.227035,0.238701,0.077808,01:14
2,0.214807,0.255969,0.082544,01:15
3,0.174884,0.267978,0.079838,01:15
4,0.145903,0.276692,0.080514,01:15
5,0.119266,0.275767,0.073748,01:15
6,0.086874,0.249827,0.064276,01:15
7,0.059052,0.233692,0.060893,01:16
8,0.052908,0.227989,0.067659,01:16
9,0.036007,0.215705,0.060893,01:16


In [ ]:
model_name = 'densenet121_pet_class.pkl'
learn.export(model_name)

## Inference with the saved model
This part is only for testing purposes. The code shown here, will be ultimately used at the WebApp server side for performing inference on user-uploaded photos.

In [ ]:
path = Path(".")
path.ls(file_exts='.pkl')
# !ls -lh models/*

(#1) [Path('densenet121_pet_class.pkl')]

In [ ]:
learn_inf = load_learner(model_name)

In [ ]:
# Extract classes from the learner
breeds = learn_inf.dls.vocab
cats = [breed.replace('_', ' ') for breed in breeds if breed[0].isupper()]
dogs = [breed.replace('_', ' ').capitalize() for breed in breeds if breed[0].islower()]
cats, dogs

(['Abyssinian',
  'Bengal',
  'Birman',
  'Bombay',
  'British Shorthair',
  'Egyptian Mau',
  'Maine Coon',
  'Persian',
  'Ragdoll',
  'Russian Blue',
  'Siamese',
  'Sphynx'],
 ['American bulldog',
  'American pit bull terrier',
  'Basset hound',
  'Beagle',
  'Boxer',
  'Chihuahua',
  'English cocker spaniel',
  'English setter',
  'German shorthaired',
  'Great pyrenees',
  'Havanese',
  'Japanese chin',
  'Keeshond',
  'Leonberger',
  'Miniature pinscher',
  'Newfoundland',
  'Pomeranian',
  'Pug',
  'Saint bernard',
  'Samoyed',
  'Scottish terrier',
  'Shiba inu',
  'Staffordshire bull terrier',
  'Wheaten terrier',
  'Yorkshire terrier'])

In [ ]:
gen, idx, probs = learn_inf.predict('images/Abyssinian_1.jpg')
gen, probs[idx]*100

('Abyssinian', TensorBase(99.9976))

In [ ]:
path = Path("images")
path.ls(file_exts='.jpg')[:10]

(#10) [Path('images/miniature_pinscher_77.jpg'),Path('images/shiba_inu_163.jpg'),Path('images/chihuahua_24.jpg'),Path('images/Maine_Coon_127.jpg'),Path('images/Ragdoll_193.jpg'),Path('images/havanese_131.jpg'),Path('images/keeshond_171.jpg'),Path('images/havanese_152.jpg'),Path('images/leonberger_62.jpg'),Path('images/Egyptian_Mau_71.jpg')]

In [ ]:
for pic in path.ls(file_exts='.jpg')[:5]:
    gen, idx, probs = learn_inf.predict(pic)
    c_gen = " ".join([w for w in gen.split("_")]).capitalize()
    print("Genre of image '{}' is {} with prob. = {:.1f}%".format(pic.name, c_gen, float(probs[idx]*100)))

Genre of image 'miniature_pinscher_77.jpg' is Miniature pinscher with prob. = 100.0%


Genre of image 'shiba_inu_163.jpg' is Shiba inu with prob. = 100.0%


Genre of image 'chihuahua_24.jpg' is Chihuahua with prob. = 99.7%


Genre of image 'Maine_Coon_127.jpg' is Maine coon with prob. = 98.2%


Genre of image 'Ragdoll_193.jpg' is Ragdoll with prob. = 100.0%
